### IMPORT LIBRARIES

In [2]:
import os

In [3]:
import numpy as np
import pandas as pd

In [4]:
import seaborn as sns
from matplotlib import pylab as plt

In [5]:
import warnings
warnings.filterwarnings('ignore')
sns.set_palette(['#0892a5','#2e9b9b','#50a290','#6fa985','#8dad7f','#a9b17e','#c4b383','#dbb68f'])

### GETTING DATA

In [6]:
files = [file for file in os.listdir('C:/Users/DMS/Downloads/ecommerce_sales_analysis/dataset')] 

In [7]:
df = pd.DataFrame()

In [8]:
for i in files:
    data = pd.read_csv('dataset/'+i)
    df = pd.concat([df,data],axis=0)

In [9]:
df.shape

(186850, 6)

### DATA CLEANING

In [10]:
# Exclude header for each dataset inside dataframe
df = df[df['Order ID'] != 'Order ID']

In [11]:
df = df.reset_index()

In [12]:
df = df.drop(columns='index')

In [13]:
df.sample(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
41395,306234,Apple Airpods Headphones,1,150,12/11/19 20:08,"908 11th St, San Francisco, CA 94016"
9326,185476,Flatscreen TV,1,300,04/25/19 18:59,"929 7th St, Los Angeles, CA 90001"
108296,165092,USB-C Charging Cable,2,11.95,03/26/19 12:25,"15 Ridge St, Atlanta, GA 30301"
90763,236001,Wired Headphones,1,11.99,07/23/19 11:39,"155 Jefferson St, Boston, MA 02215"
186286,259160,Lightning Charging Cable,1,14.95,09/30/19 09:43,"402 2nd St, Los Angeles, CA 90001"
31477,296757,Apple Airpods Headphones,1,150,12/24/19 18:46,"80 Hill St, Boston, MA 02215"
125593,199182,AAA Batteries (4-pack),1,2.99,05/22/19 20:17,"32 8th St, San Francisco, CA 94016"
39296,304227,Wired Headphones,1,11.99,12/08/19 17:16,"760 Main St, Los Angeles, CA 90001"
44721,309420,34in Ultrawide Monitor,1,379.99,12/06/19 23:00,"999 6th St, Dallas, TX 75001"
27653,245587,27in 4K Gaming Monitor,1,389.99,08/14/19 16:18,"34 13th St, Austin, TX 73301"


In [14]:
# Check for data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186495 entries, 0 to 186494
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          185950 non-null  object
 1   Product           185950 non-null  object
 2   Quantity Ordered  185950 non-null  object
 3   Price Each        185950 non-null  object
 4   Order Date        185950 non-null  object
 5   Purchase Address  185950 non-null  object
dtypes: object(6)
memory usage: 8.5+ MB


In [15]:
# Check null values
df.isna().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [16]:
# Check null values
df[df.isna().any(axis=1)]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN
734,NaN,NaN,NaN,NaN,NaN,NaN
1430,NaN,NaN,NaN,NaN,NaN,NaN
1550,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
184825,NaN,NaN,NaN,NaN,NaN,NaN
185087,NaN,NaN,NaN,NaN,NaN,NaN
185690,NaN,NaN,NaN,NaN,NaN,NaN
186196,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Drop null vales
df = df.dropna()

In [18]:
# Check null values
df.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64